<a href="https://colab.research.google.com/github/kajian1008/Whisper_test/blob/main/Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q torchaudio

SAMPLING_RATE = 16000

import torch
torch.set_num_threads(1)

from IPython.display import Audio
from pprint import pprint

In [ ]:
USE_ONNX = False # change this to True if you want to test onnx model
if USE_ONNX:
    !pip install -q onnxruntime

model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=True,
                              onnx=USE_ONNX)

(get_speech_timestamps,
 save_audio,
 read_audio,
 VADIterator,
 collect_chunks) = utils

In [ ]:
import os
directory = '/content/drive/MyDrive/ja_voice_finetuning/clips'
out_directory = '/content/drive/MyDrive/ja_voice_finetuning/silero-clips'

# Create the output directory if it doesn't exist
os.makedirs(out_directory, exist_ok=True)

# 指定されたディレクトリ内の全てのファイルをループ処理
for filename in os.listdir(directory):
    if filename.endswith(".mp3"):  # ファイルが.mp3で終わる場合
        file_path = os.path.join(directory, filename)  # ファイルのフルパスを取得
        print(f'Processing file: {file_path}')

        # read_audio とは、ファイルを読み込んで適切な形式（例: numpy array）に変換する関数です
        audio = read_audio(file_path, sampling_rate=SAMPLING_RATE)

        # get_speech_timestamps は、音声のタイムスタンプを検出する関数です
        speech_timestamps = get_speech_timestamps(audio, model, sampling_rate=SAMPLING_RATE)

        if speech_timestamps:  # タイムスタンプリストが空でない場合
            # 処理された音声を保存する関数を呼び出す
            output_path = os.path.join(out_directory, filename)
            save_audio(output_path, collect_chunks(speech_timestamps, audio), sampling_rate=SAMPLING_RATE)
        else:
            print(f"No speech detected in file: {file_path}")

In [ ]:
!pip install pydub openai-whisper
!pip install kora

In [ ]:
from kora.xattr import get_id
def get_gdrive_link(file_path):
  fid = get_id(file_path)
  print(f"https://drive.google.com/file/d/{format(fid)}/view")
  return f"https://drive.google.com/file/d/{format(fid)}/view"

In [ ]:
import whisper
import torch
import os
import csv  # CSVファイル出力のために必要

# 現在利用可能なデバイスを確認
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Whisperモデルをロード
model = whisper.load_model("turbo", device=device)

# 出力先のCSVファイルパスを定義
output_csv_file = '/content/drive/MyDrive/ja_voice_finetuning/train0.csv'


# CSVファイルを開き、結果を書き込む
with open(output_csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # CSVのヘッダーを書き込む
    writer.writerow(['url', 'path','sampling_rate', 'correct', 'whisper'])

    # 指定されたディレクトリ内の全てのファイルをループ処理
    for filename in os.listdir(out_directory):
        if filename.endswith(".mp3"):  # ファイルが.mp3で終わる場合
            file_path = os.path.join(out_directory, filename)  # ファイルのフルパスを取得
            print(f'Processing file: {file_path}')
            result = model.transcribe(file_path, language="ja")
            # 結果をCSVに書き込む（ここではurlとcontentは空とする）
            # https://drive.google.com/file/d/1ISt6Pg2eJFuHvV1Fkm2NZBvzAJ4grewq/view?usp=drive_link
            writer.writerow([get_gdrive_link(file_path), file_path, SAMPLING_RATE ,'', result["text"]])
            print(result["text"])

print(f"Processing complete. Results saved to {output_csv_file}")